# Stacking fault energy maps

This Notebook follows the workflow for computing the 2D stacking fault energy map, i.e. gamma surface, for a variety of crystallographic planes.

__Global workflow order:__ 

This Notebook requires calculation_crystal_space_group records and the "unique_crystals.csv" file, both of which are generated by the "3. Crystal relaxation" Notebook.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# https://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database

### Load database

In [2]:
database = iprPy.load_database('potential_testing')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

15


### Specify prototype-composition delimiters

In [4]:
proto_comps = {}
proto_comps['A1--Cu--fcc'] = ['Ne', 'Al', 'AlS', 'Ar', 'Ca', 'Fe', 'FeS', 'Ni', 'Cu', 'CuS', 
                              'Kr', 'Sr', 'Rh', 'Pd', 'Ag', 'Xe', 'Ir', 'Pt', 'Au', 'Pb', 
                              'Ac', 'Ce', 'Yb', 'Th']
proto_comps['A2--W--bcc'] = ['Li', 'Na', 'K', 'V', 'Cr', 'Fe', 'FeS', 'Rb', 'Nb', 'Mo', 
                             'Cs', 'Ba', 'Ta', 'W', 'Eu']
proto_comps['A3--Mg--hcp'] = ['H', 'He', 'Be', 'Mg', 'MgS', 'Sc', 'Ti', 'Co', 'Zn', 'Y', 
                              'Zr', 'Tc', 'Ru', 'Cd', 'Hf', 'Re', 'Os' 'Tl', 'Gd', 'Tb', 
                              'Dy', 'Ho', 'Er', 'Tm', 'Lu']
proto_comps['A4--C--dc'] = ['C', 'Si', 'SiS', 'Ge', 'Sn']

### Limit unique crystals using prototype-composition delimiters

In [5]:
proto_parents = {}
unique_crystal_keys = []
for prototype in proto_comps:
    proto_parents[prototype] = unique_crystals[(unique_crystals.prototype == prototype)
                                              &(unique_crystals.composition.isin(proto_comps[prototype]))].calc_key.tolist()
    print(len(proto_parents[prototype]), prototype)
    unique_crystal_keys.extend(proto_parents[prototype])
print(len(unique_crystal_keys), 'total')

1 A1--Cu--fcc
0 A2--W--bcc
0 A3--Mg--hcp
0 A4--C--dc
1 total


## 1. stacking_fault_map_2D calculation

In [6]:
calculation = iprPy.load_calculation('stacking_fault_map_2D')
run_directory = iprPy.load_run_directory('potential_testing_1')

### Build input_dict

In [7]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'stackingfault_file', 'stackingfault_content', 'stackingfault_family', 'stackingfault_cutboxvector', 'stackingfault_faultpos', 'stackingfault_shiftvector1', 'stackingfault_shiftvector2', 'stackingfault_numshifts1', 'stackingfault_numshifts2', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [8]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information from crystal_space_group results
buildcombos                 atomicparent load_file parent
parent_record               calculation_crystal_space_group
parent_load_key             unit-cell-atomic-system

# Build defect records
buildcombos                 defect stackingfault_file
defect_record               stacking_fault

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   3 3 8

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
stackingfault_numshifts1    21
stackingfault_numshifts2    21
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [9]:
# Parse input script
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

In [10]:
input_dict['parent_name'] = unique_crystal_keys

### Prepare calculations

In [11]:
database.prepare(run_directory, calculation, **input_dict)

In [12]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\potential_testing :
- 1 of style calculation_stacking_fault_map_2D
 - 0 are complete
 - 1 still to run
 - 0 issued errors


### Run calculations

In [13]:
database.runner(run_directory)

Runner started with pid 10412
No simulations left to run


In [13]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
errors = []
for error in error_df.error:
    lines = error.splitlines()
    err = ''
    for i in range(len(lines)-1, -1, -1):
        if 'Error:' in lines[i]:
            err = '\n'.join(lines[i:-1])
            break
        if i == 0:
            err = error
    errors.append(err)
for error in np.unique(errors):
    print(error)

0 calculations issued errors:


## 2. Calculation analysis

In [14]:
records = database.get_records_df(style=calculation.record_style, status='finished')
iprPy.analysis.assign_composition(records, database)
pot_records = database.get_records_df(style='potential_users_LAMMPS')

In [15]:
records.keys()

Index(['LAMMPS_version', 'energytolerance', 'error', 'family',
       'forcetolerance', 'gammasurface', 'iprPy_version', 'key', 'load_file',
       'load_options', 'load_style', 'maxatommotion', 'maxevaluations',
       'maxiterations', 'numshifts1', 'numshifts2', 'potential_LAMMPS_id',
       'potential_LAMMPS_key', 'potential_id', 'potential_key', 'script',
       'shiftvector1', 'shiftvector2', 'sizemults', 'stackingfault_id',
       'stackingfault_key', 'status', 'symbols', 'composition'],
      dtype='object')

In [16]:
# Settings
outputpath = 'C:/Users/lmh1/Documents/potential_testing_results'
surface_energy_units = 'mJ/m^2'

In [17]:
data = []
for record in records.itertuples():
    crystal = unique_crystals[unique_crystals.calc_key == record.load_file.split('.')[0]].iloc[0]
    
    dat = {}
    dat['calc_key'] = record.key
    dat['potential_LAMMPS_key'] = crystal.potential_LAMMPS_key
    dat['potential_LAMMPS_id'] = crystal.potential_LAMMPS_id
    dat['potential_key'] = crystal.potential_key
    dat['potential_id'] = crystal.potential_id
    dat['prototype'] = crystal.prototype
    dat['family'] = crystal.family
    dat['composition'] = record.composition
    dat['a'] = crystal.a
    dat['b'] = crystal.b
    dat['c'] = crystal.c
    dat['alpha'] = crystal.alpha
    dat['beta'] = crystal.beta
    dat['gamma'] = crystal.gamma
    dat['gammasurface'] = record.gammasurface
    dat['stackingfault_id'] = record.stackingfault_id
    dat['stackingfault'] = record.stackingfault_id.replace(crystal.prototype+'--', '').replace('sf', '')
    dat['shiftvector1'] = '[' + record.shiftvector1 + ']'
    dat['shiftvector2'] = '[' + record.shiftvector2 + ']'
    data.append(deepcopy(dat))
    
columns = ['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id',
           'composition', 'prototype', 'family', 
           'a', 'b', 'c', 'alpha', 'beta', 'gamma',
           'stackingfault_id', 'stackingfault', 'gammasurface', 'shiftvector1', 'shiftvector2']

data = pd.DataFrame(data, columns=columns).sort_values(['potential_LAMMPS_id', 'composition', 'stackingfault_id'])

In [18]:
data

,calc_key,potential_LAMMPS_key,potential_LAMMPS_id,potential_key,potential_id,composition,prototype,family,a,b,c,alpha,beta,gamma,stackingfault_id,stackingfault,gammasurface,shiftvector1,shiftvector2
0,c982bf3c-4f43-4719-9d7d-a5761035c014,7cf8ea1e-859d-4878-bf35-0b660ba5aafc,1999--Mishin-Y--Al--LAMMPS--testing1,eb33d631-9ac6-4b51-8642-7a92462b96a3,1999--Mishin-Y--Al,Al,A1--Cu--fcc,A1--Cu--fcc,4.050005,4.050005,4.050005,90.0,90.0,90.0,A1--Cu--fcc--111sf,111,<atomman.defect.GammaSurface.GammaSurface obje...,[0.5 -0.5 0.0],[0.5 0.5 -1.0]


### Generate plots

In [19]:
for implememtation_key in np.unique(records.potential_LAMMPS_key):
    imp_results = data[data.potential_LAMMPS_key == implememtation_key]
    potential = imp_results.iloc[0].potential_id
    implementation = imp_results.iloc[0].potential_LAMMPS_id
    
    contentpath = os.path.join(outputpath, potential, implementation)
    if not os.path.isdir(contentpath):
        os.makedirs(contentpath)
    
    for series in imp_results.itertuples():
        key = series.calc_key
        gammasurface = series.gammasurface
        
        # Generate 2D surface
        gammasurface.E_gsf_surface_plot(energyperarea_unit=surface_energy_units)
        plt.savefig(os.path.join(contentpath, key + '.sf.2D.png'), bbox_inches='tight')
        plt.close()
        
        # Generate 1D plots
        gammasurface.E_gsf_line_plot(vect=gammasurface.a1vect, energyperarea_unit=surface_energy_units)
        plt.savefig(os.path.join(contentpath, key + '.sf.a1.png'), bbox_inches='tight')
        plt.close()
        gammasurface.E_gsf_line_plot(vect=gammasurface.a2vect, energyperarea_unit=surface_energy_units)
        plt.savefig(os.path.join(contentpath, key + '.sf.a2.png'), bbox_inches='tight')
        plt.close()
        
        # Save data model
        with open(os.path.join(contentpath, key + '.sf.json'), 'w') as f:
            gammasurface.model().json(fp=f, indent=4)


### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [20]:
# Add calculation data to PotentialProperties records
for implememtation_key in np.unique(data.potential_LAMMPS_key):
    imp_records = data[data.potential_LAMMPS_key == implememtation_key].sort_values('family')
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation_id = pot_record.id
    potential_id = pot_record.pot_id
    potential_key = pot_record.pot_key
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['stacking-faults'] = fs_model = DM()
    
    for composition in np.unique(imp_records.composition):
        comp_records = imp_records[imp_records.composition == composition]
        comp_model = DM()
        comp_model['composition'] = composition
        
        for prototype in np.unique(comp_records.prototype):
            proto_records = comp_records[comp_records.prototype == prototype]
            proto_model = DM()
            proto_model['prototype'] = prototype
            
            for alat in np.unique(proto_records.a):
                alat_records = proto_records[proto_records.a == alat]
                alat_model = DM()
                alat_model['a'] = alat
                
                for series in alat_records.sort_values('stackingfault').itertuples():
                    plot = DM()
                    plot['name'] = series.stackingfault + ' 2D'
                    plot['file'] = series.calc_key + '.sf.2D.png'
                    alat_model.append('plot', plot)
                    
                    plot = DM()
                    plot['name'] = series.stackingfault + ' ' + series.shiftvector1
                    plot['file'] = series.calc_key + '.sf.a1.png'
                    alat_model.append('plot', plot)
                    
                    plot = DM()
                    plot['name'] = series.stackingfault + ' ' + series.shiftvector2
                    plot['file'] = series.calc_key + '.sf.a2.png'
                    alat_model.append('plot', plot)
                    
                    model = DM()
                    model['file'] = series.calc_key + '.sf.json'
                    alat_model.append('model', model)
                    
                proto_model.append('alats', alat_model)
            comp_model.append('prototypes', proto_model)
        fs_model.append('compositions', comp_model)
    
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())